In [1]:
import sys, os

#temporary 
sys.path.append("../../../learnMSA")

sys.path.append("../../bin")
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from eval_model_class import PredictionGTF
from models import lstm_model, custom_cce_f1_loss

2024-11-02 10:19:49.612809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 10:19:49.704534: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 10:19:50.139695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/beckerf/mambaforge/envs/learnMSAdev/lib/:/home/beckerf/mambaforge/envs/learnMSAdev/lib/python3.10/site-packages/nvidia/cudnn/lib:
2024-11-02 10:19:50.139929: W tensorflow/stream_executor/platform/default/ds

In [2]:
batch_size = 20
seq_len = 9999
strand = '+'

emb=False
hmm_factor=1
trans_lstm=False

inp_data_dir = 'inp/'
out_dir = 'test_train/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

genome_path = f'{inp_data_dir}/genome.fa'
annot_path= f'{inp_data_dir}/annot.gtf'


pred_gtf = PredictionGTF( 
     seq_len=seq_len, 
    batch_size=batch_size,
    hmm=True, 
    emb=False, 
    num_hmm=1,
    hmm_factor=1,
    genome_path=genome_path,
    annot_path=annot_path, 
    softmask=True, strand=strand,
)

# load input data x_seq 
x_seq, y_seq, _ = pred_gtf.load_inp_data(    
    strand=strand, 
    chunk_coords=True, softmask=True, pad=False
)


In [3]:
# see lstm_model documentation for more arguments 
config = {
    "num_epochs": 10,
    "stride": 0,
    "units": 100,
    "filter_size": 32,
    "numb_lstm": 2,
    "numb_conv": 3,
    "dropout_rate": 0.0,
    "hmm_dense": 32,
    "pool_size": 9,
    "lr": 1e-4,
    "batch_size": batch_size,
    "w_size": seq_len,
    'output_size': 15,
}

relevant_keys = ['units', 'filter_size', 'kernel_size', 
                     'numb_conv', 'numb_lstm', 'dropout_rate', 
                     'pool_size', 'stride',  
                     'output_size', 'multi_loss']

relevant_args = {key: config[key] for key in relevant_keys if key in config}
model = lstm_model(**relevant_args)
adam = Adam(learning_rate=config["lr"])
f1loss = custom_cce_f1_loss(2, batch_size=config["batch_size"])


model.compile(loss=f1loss, optimizer=adam, metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, None, 6)]    0           []                               
                                                                                                  
 initial_conv (Conv1D)          (None, None, 32)     608         ['main_input[0][0]']             
                                                                                                  
 layer_normalization1 (LayerNor  (None, None, 32)    64          ['initial_conv[0][0]']           
 malization)                                                                                      
                                                                                                  
 conv_1 (Conv1D)                (None, None, 32)     9248        ['layer_normalization1[0][0

In [4]:
model.fit(x=x_seq, y=y_seq, 
          epochs=config["num_epochs"], 
          batch_size=config["batch_size"])

Epoch 1/10


2024-11-02 10:19:57.989283: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8800
2024-11-02 10:19:58.078484: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-02 10:19:58.080607: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-02 10:19:58.080633: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-11-02 10:19:58.081679: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-02 10:19:58.081760: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-11-02 10:19:58.127537: I tensorflow/stream_executor/cuda/c

115/212 [===============>..............] - ETA: 11s - loss: 3.2727 - accuracy: 0.7405